# Agenda, day 2

1. Recap + Q&A
2. Magic methods
3. Class attributes
4. Finding attributes with ICPO
5. Inheritance -- what it is, and how ICPO influences it
6. Three models for method inheritance
7. Data inheritance
8. What next?

# Recap

How do we solve problems? Often, it's easiest if we can use a data structure that's appropriate for the problem we're trying to solve. Objects allow us to create a data structure that is specifically designed to solve our problem, or to make it easier to think about our problem. Even if it's just a wrapper around strings, lists, tuples, dicts, etc., often the fact that an object has a different name can help us to solve our problems more easily.

The idea of object-oriented programming is: Create new data structures that are easier to use and think about than existing data structures.

- By putting these core data structures inside of an object, we can think at a higher level -- abstraction
- Methods (verbs) are defined on the class, which means that they are tightly bound to a particular type of data. Regular functions can be invoked with *any* type of value as an argument, although that might not be appropriate, and you can get an error. Methods are only available on the objects for which they actually exist. If you try to run a method on an object where it wasn't defined, you'll get an "attribute error," meaning that the name after the `.` doesn't exist.
- We define new "classes," data structures/types. A type is a class, and a class is a type -- at least when we're talking about them. The `type` function, which tells us what class something has, can only be used there. The `class` keyword can only be used when introducing a new data structure to Python.
- A class is also a factory for new objects of that type. If I have a `Person` class, then I can create new instances of `Person` by invoking the class, `Person()`. (Maybe I need to pass one or more arguments.) We often say that a class is a "factory object," because it creates new instances.
- The most important method in a class is `__init__` ("dunder init") whose job is to add new attributes to the object just after it's created, but before it's returned to the caller. `__init__` is invoked automatically by `__new__`, which we never rewrite, but which creates the new object. If you create 5 instances of `Person`, then `__init__` will be invoked 5 times, once for (and on) each instance.
- Each time we invoke a method, the first argument is automatically set to the instance itself. If I call `a.b()`, then the method `b` will get `a` as its first argument. However, the parameter into which `a` will be assigned is always called `self`. That parameter is automatically populated, but it is important.
- The whole idea of `self` is that it refers to the current instance. Other languages often have a special name, often `this`, which refers to the current object. Python is unusual in that `self` is a local variable, a parameter, just like any other local variable or parameter. It has the same priority, no magic needed, as any other local variable.
- When we assign to an attribute on `self`, we are adding an attribute to our object. (Unless it already exists, in which case we're updating the value for that attribute.) This storage on the object sticks around until the object goes away. If you set an attribute in `__init__` on `self`, that attribute exists on that object for the rest of the Python session, unless you delete the object or the attribute.
- In theory, you could assign all attributes to the object outside of `__init__`. However, it's easiest and best to assign attributes inside of `__init__`, so that we can keep track of things more easily. When you want to maintain someone else's class definition, it's easiest to do so if you have a clear list of all attributes.
- Most attributes are set based on the parameters, which get their values from arguments to the class. But you can set attributes without any relationship to those parameters, if you want.



In [1]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0

    def greet(self):
        return f'Hello, {self.name}!'

p = Person('Reuven', 46)        

In [2]:
p.greet() 

'Hello, Reuven!'

In [3]:
p.name

'Reuven'

In [4]:
p.name = 'whoever'

In [5]:
p.greet()

'Hello, whoever!'

# Memory management

In Python, we have two ways in which the language handles memory issues:

1. Reference counting -- when there are 0 references to an object (e.g., variable names, references from lists/dicts/tuples), then the object is immediately erased, and its memory is freed up.
2. Garbage collection -- if we have a circular data structure, then there will always be at least one reference to it, and it won't get released. Python occasionally looks for these and releases the "islands" of memory

When is an object removed? We see, if it's not a circular island in memory, then whenever the last variable or other reference goes away.

Practically speaking, this means that if you have a global variable that refers to an object, then the object will stick around until Python shuts down.

# Methods

Normally, methods are

    - defined on the class (in the class body, with `def`)
    - invoked on the instance

But actually, Python does some rewriting here! If I say

    an_instance.a_method()

Python rewrites this to be

    a_class.a_method(an_instance)

In [6]:
s = 'abcde'

s.upper()   # I'm invoking the "str.upper" method via the instance

'ABCDE'

In [7]:
# I can equivalently say:

str.upper(s) # this is what Python wrote line 3 in the above cell to be

'ABCDE'

Now you can see how `self` is populated! The method call is rewritten, and the first argument becomes the instance.

Weak references -- these allow a variable to refer to a value without modifying its reference count. In this way, you can refer to a value without stopping it from being garbage collected.

# Exercise: Cellphone

1. Define a `Cellphone` class. Each instance will have two attributes:
    - `number`
    - `model`
2. You should be able to invoke the `call` method on your instance of `Cellphone`. It will return a string saying, "Calling..." and print the number that it is calling.

In [8]:
class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model

c1 = Cellphone('12345', 'iPhone')
c2 = Cellphone('67890', 'Samsung')

In [9]:
vars(c1)

{'number': '12345', 'model': 'iPhone'}

In [10]:
vars(c2)

{'number': '67890', 'model': 'Samsung'}

In [11]:
c1.number

'12345'

In [12]:
c2.number

'67890'

In [13]:
c1.model

'iPhone'

In [14]:
c2.model

'Samsung'

In [15]:
# let's now add the "call" method

class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model
    def call(self):
        return f'Calling {self.number} on your {self.model}...'

c1 = Cellphone('12345', 'iPhone')
c2 = Cellphone('67890', 'Samsung')

print(c1.call())
print(c2.call())

Calling 12345 on your iPhone...
Calling 67890 on your Samsung...


In [16]:
# let's now add the "call" method

class Cellphone:
    def __init__(self, number, model):
        self.number = number
        self.model = model
    def call(self, other_number):
        return f'Calling {other_number} from your {self.model}, with a number of {self.number}...'

c1 = Cellphone('12345', 'iPhone')
c2 = Cellphone('67890', 'Samsung')

print(c1.call('1111'))
print(c2.call('22222'))

Calling 1111 from your iPhone, with a number of 12345...
Calling 22222 from your Samsung, with a number of 67890...


# Single Responsibility Principle

This is a big deal in the world of objects -- the idea is that one class should do one thing, not many things. If you find yourself loading lots of functionality onto a particular class, that might mean you need to divide it into multiple classes.

# Magic methods

When we perform certain operations in Python, the language looks for a method with a specific name that implements that operation. These methods all start and end with a double underscore, `__`. They are also known as "dunder methods," but many people call them "magic methods." You almost never want to call a magic method directly! Rather, let Python invoke the method on your behalf at the appropriate time.

The point of magic methods is that we can make our objects adhere to Python conventions and expectations, and behave much like the core builtin objects -- str, list, tuple, dict.

There are about 100 magic methods, and many of them are for complex parts of Python that we're not going to cover here. 

# Magic method: `__len__`

If you call `len` (a builtin function) on an object, then Python looks for a `__len__` method on that object, and invokes it. If there is no `__len__` method, then we get an error, indicating that the "object has no len".

It's very very very very tempting to just invoke `__len__` directly, rather than invoking the `len` function which will invoke `__len__` behind the scenes. But don't do it!

In [17]:
p = Person('Reuven', 46)

In [18]:
len(p)

TypeError: object of type 'Person' has no len()

# What happened?

1. I invoked `len(p)`
2. `len` looks for `p.__len__`. It didn't find such a method.
3. It then raised an exception, saying that it didn't find such a method.

We could, of course, add a `__len__` method to `Person`. By doing that, we would be defining what it means to have a `len`. But it's totally OK to say that certain objects have no length -- such as integers.

In [27]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0

    def greet(self):
        return f'Hello, {self.name}!'

p = Person('Reuven', 46)        
len(p)   # this produces an error

TypeError: object of type 'Person' has no len()

In [28]:
class Person:
    def __init__(self, name, shoe_size):
        self.name = name
        self.shoe_size = shoe_size
        self.bank_balance = 0

    def greet(self):
        return f'Hello, {self.name}!'

    def __len__(self):
        return self.shoe_size   # we'll say that when we want the person's len, we really want their shoe size

p = Person('Reuven', 46)        

In [29]:
len(p)  # this led to invoking p.__len__(), and we get the result back -- so long as __len__ returns an integer, we're fine

46

# What does `len` do?

The answer is: Whatever the `__len__` method on a class tells it to do:

- `str.__len__` returns the number of characters
- `list.__len__` and `tuple.__len__` both return the number of elements
- `dict.__len__` returns the number of key-value pairs

You can define `__len__` to return the same value each time, a random value each time, or (more realistically) a value that depends on the attributes in the object.

# Exercise: How many scoops?

On Monday, we defined `Scoop` and `Bowl` classes, for ice cream. We're going to keep using them today. 

1. Now, modify `Bowl` such that we can invoke `len` on it. We should get back the number of scoops in `bowl.scoops`.
2. Use this method twice, once before adding scoops, and once after.

In [32]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def add_scoops(self, *new_scoops):   # *new_scoops means new_scoops will be a tuple with all positional arguments
        for one_scoop in new_scoops:
            self.add_scoop(one_scoop)   # we can invoke a method on our own!

    def __len__(self):
        return len(self.scoops)

b = Bowl()
b.add_scoops(s1, s2, s3)

print(b.flavors())
print(len(b))

['chocolate', 'vanilla', 'coffee']
3


In [33]:
len(b.scoops)

3

In [34]:
# IN

my_list = [10, 20, 30, 40, 50]
length_of_list = len(my_list)
print(length_of_list)

5


In [35]:
print(len(my_list))

5


In [36]:
# don't do this!  but you could say

b.__len__()

3

In [37]:
s = 'abcde'
s.__len__()

5

In [38]:
# can you assign attributes and methods outside of the class, after the class has been written?

In [52]:
class MyClass:
    def __init__(self, x):
        self.x = x

    def by_3(self):
        return self.x * 3

    def return_vars(self):
        return vars(self)

m = MyClass(10)  # I passed 10, which was passed to x in __init__, which means that self.x was set to 10

In [53]:
vars(m)

{'x': 10}

In [54]:
# self (local variable) inside of __init__ and m (global variable) outside of __init__ are the same object
# modifying one affects the other

In [55]:
m.x

10

In [56]:
m.x = 20
vars(m)

{'x': 20}

In [57]:
m.by_3()   # when we invoke the method, m (global) is assigned to self (local)

60

In [58]:
# can I add a new attribute to an object from outside of __init__, or the class itself?
# YES, this is Python! We can do anything, even if it's a bad idea!

m.y = 999

In [59]:
m.return_vars()

{'x': 20, 'y': 999}

In [60]:
# can I add a new method to the class outside of its definition?
# yes... but it's a very very very bad idea, and it's also a bit complex to do

# Next up

1. `__str__` and `__repr__`
2. More magic methods, and what they do (and how they work)


When we invoke `print` on an object, what happens?

- `print` invokes `str` on whatever argument it got. That returns a string based on the value. This is how `print` can display any Python value!
- What happens when we invoke `str`? 

In [61]:
class MyClass:
    def __init__(self, x):
        self.x = x

m = MyClass(10)

print(m)  # what is this printing? 

When we invoke `print` on an object, it invokes `str`. When `str` is invoked on an object, it looks for and invokes the `__str__` method on that object. By default, `__str__` gives us a very ugly result (that works).

If we want, we can define our own version of `__str__`. So long as it returns a string, we can return anything we want.

In [63]:
class MyClass:
    def __init__(self, x):
        self.x = x

    def __str__(self):
        return f'An instance of MyClass with x = {self.x}'

m = MyClass(10)

print(m)  # print(m) -> print(str(m)) -> print(m.__str__())

An instance of MyClass with x = 10


It's almost always a good idea to implement `__str__` on an object.

(I'm going to contradict that in a few minutes.)

# Exercise: Printable scoops

Right now, if you invoke `print` on any of our `Scoop` instances, you'll get an ugly default string back. Add a `__str__` method to `Scoop`, such that it says, `'Scoop of FLAVOR'`.

In [65]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def add_scoops(self, *new_scoops):   # *new_scoops means new_scoops will be a tuple with all positional arguments
        for one_scoop in new_scoops:
            self.add_scoop(one_scoop)   # we can invoke a method on our own!

    def __len__(self):
        return len(self.scoops)

b = Bowl()
b.add_scoops(s1, s2, s3)

print(b.flavors())
print(len(b))
print(s1)
print(s2)
print(s3)

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee


# Exercise: Printable bowls

If I run `str` on an instance of `Bowl`, or if I `print` an instance of `Bowl`, I want to get back a string:

- Goes through each scoop, printing it
- Numbers those scoops, as well
- Has a short header saying, "Bowl with scoops"

In other words, if I say

    print(b)

I should see something like:

    Bowl with:
        1. Scoop of chocolate
        2. Scoop of vanilla
        3. Scoop of coffee


In [75]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __str__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def add_scoops(self, *new_scoops):   # *new_scoops means new_scoops will be a tuple with all positional arguments
        for one_scoop in new_scoops:
            self.add_scoop(one_scoop)   # we can invoke a method on our own!

    def __len__(self):
        return len(self.scoops)

    def __str__(self):
        output = 'Bowl with:\n'
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'  {index}: {one_scoop}\n'
        
        return output

b = Bowl()
b.add_scoops(s1, s2, s3)

print(b.flavors())
print(len(b))
print(s1)
print(s2)
print(s3)
print('***')
print(b)

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee
***
Bowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee



JB: How can I use AI to improve my learning?

- https://speakerdeck.com/reuven/edu-summit-2025-dot-key
- https://www.youtube.com/watch?v=rEddzOVCAEM&list=PLbFHh-ZjYFwGbJ88vVV1MqusPfbLxtXFu&index=8

In [76]:
# if we print our bowl, we get this:

print(b)

Bowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee



In [77]:
# in Jupyter, I can just put any expression in a cell, and see the printed representation

b

# `__str__` and `__repr__`

There are actually *two* magic methods that govern how an object displays itself, or turns itself into a string.

- `__str__` is the one we normally think about, because it affects end users, and it naturally aligns with `str`. The output from `__str__` is supposed to be for end users.
- `__repr__`, the printed representation, is meant for coders and others working behind the scenes -- in Jupyter, a debugger, and elsewhere. In theory, according to the Python conventions, the string returned by `__repr__` should be legitimate Python code.

Realistically, I almost never want to distinguish between `__str__` and `__repr__`.  Moreover:

- If you define both of them, they'll be displayed in different contexts
- If you define only `__str__`, then it'll be showed for `print` but not in Jupyter
- If you define only `__repr__`, it'll be used in all contexts

I argue: Always define `__repr__`, never define `__str__`... unless you want to show different things to internal developers and end users.

In [78]:
# change Scoop and Bowl to define __repr__, and not __str__

class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')

class Bowl:
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def add_scoops(self, *new_scoops):   # *new_scoops means new_scoops will be a tuple with all positional arguments
        for one_scoop in new_scoops:
            self.add_scoop(one_scoop)   # we can invoke a method on our own!

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'  {index}: {one_scoop}\n'
        
        return output

b = Bowl()
b.add_scoops(s1, s2, s3)

print(b.flavors())
print(len(b))
print(s1)
print(s2)
print(s3)
print('***')
print(b)

['chocolate', 'vanilla', 'coffee']
3
Scoop of chocolate
Scoop of vanilla
Scoop of coffee
***
Bowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee



In [79]:
s1

Scoop of chocolate

In [80]:
b

Bowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee

# What other magic methods are there?

- When you compare `a == b`, this actually invokes the `__eq__` magic method. The expression is rewritten to be `a.__eq__(b)`.
- When you retrieve `a[b]`, this actually invokes the `__getitem__` magic method. This is rewritten to be `a.__getitem__(b)`.
- When you add `a + b`, this actually invokes the `__add__` magic method. The expression is rewritten to be `a.__add__(b)`.

These are all "callback" methods, meaning that we aren't supposed to define them ourselves. Rather, when we define them, we're putting them in place for Python to invoke them at particular times.

In [82]:
11 | 21   # this does bitwise "or"

31

In [83]:
# the | operator uses a magic method, like all operators. That's why sets, dicts, and NumPy/Pandas data sets can use |
# they simply defined the right magic method on their objects

In [85]:
class MyClass:
    def __init__(self, x):
        self.x = x

m1 = MyClass(10)
m2 = MyClass(10)

In [86]:
m1 == m2  

False

In [87]:
class MyClass:
    def __init__(self, x):
        self.x = x

    def __eq__(self, other):
        return self.x == other.x

m1 = MyClass(10)
m2 = MyClass(10)

m1 == m2   # m1.__eq__(m2)

True

# Next up:

1. Class attributes
2. Attribute lookup
3. Inheritance

# Class attributes

To understand class attributes, we're going to tell a little story. The story is that our company produces the `Person` class. We have many customers who are paying top dollar for our class.

In [88]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1
Hello, name2


Our customers have demanded that we add functionality to this class. They want to know how many instances of `Person` we have created at a time.

My plan: I know that every time we create a new instance, we invoke `__init__`. Every time we invoke `__init__`, I'll add 1 to a global variable, `population`, that keeps track of this stuff.

In [89]:
population = 0

class Person:
    def __init__(self, name):
        self.name = name
        population += 1  # because we're assigning to population here, it's a local variable, unconnected to the global population!

    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, population = {population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {population}')

print(p1.greet())
print(p2.greet())

Before, population = 0


UnboundLocalError: cannot access local variable 'population' where it is not associated with a value

In [90]:
# one fix: use the "global" statement in our method 
# that tells Python not to create a local variable, but always to use the global

population = 0

class Person:
    def __init__(self, name):
        global population
        self.name = name
        population += 1 

    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, population = {population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1
Hello, name2


# Everything is an object

Remember that absolutely everything in Python is an object. And remember that every object has attributes. And remember that we can set/retrieve attributes on any object we want.

`Person` is a class. But it is also an object, because everything is an object.

Because `Person` is an object, it has attributes. We can set attributes.

So far, we have been setting attributes on the *instances* of `Person`. Now we're going to set an attribute on the class itself, on the factory object.

How do we do this? We assign to an attribute on the class, that's all.

In [91]:
# much better: add an attribute to Person

class Person:
    def __init__(self, name):
        self.name = name
        Person.population += 1 

    def greet(self):
        return f'Hello, {self.name}'

# after defining the Person class, we can say
Person.population = 0

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
Hello, name1
Hello, name2


We can do even better than this!

Anything defined *inside* of the class definition is actually an attribute on the class, *not* a variable. When we define `__init__` and `greet` inside of the `class` block, we're defining `Person.__init__` (an attribute) and `Person.greet` (another attribute).

In [92]:
s

'abcde'

In [93]:
str.upper(s)  # the str.upper method is an attribute on str

'ABCDE'

In [ ]:
# best of all: define population as an attribute *inside* of the class body!
# when we do this, we don't preface it with a . or with the class name -- beacuse the class hasn't yet been defined!

class Person:
    population = 0   # this is still defining Person.population, just in a more standard way

    def __init__(self, name):
        self.name = name
        Person.population += 1 

    def greet(self):
        return f'Hello, {self.name}'


print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')

print(p1.greet())
print(p2.greet())

# Class attributes

Every object in Python has attributes. When we assign to `self`, we are assigning to attributes on our instances. But when we assign to `Person`, we're adding an attribute to the class object.

### Where and why do we want class attributes?

1. It allows us to have shared information/state. Here, we're sharing `population`. It could be a common resource, such as a bank account, that all instances are sharing.
2. It gives us a sort of constant that all instances can use easily. Instead of defining a value in a global variable, or hard-coding it in our code, we can just use a class attribute.

In [94]:
Person.population

2

In [95]:
Person.population = 0

In [96]:
Person.population

0

# Exercise: Limited size bowls

So far, we've been able to add any number of scoops to our bowls. But the times have changed! 

1. Modify the `add_scoops` method on `Bowl` such that the maximum number of scoops you can have in the bowl is 3. No matter how many times you invoke `add_scoops` and how many arguments you give it, you should have a max of 3.
2. Any extra scoops are just ignored.
3. Don't hard-code that as a 3, but set a class attribute, `MAX_SCOOPS`, which you can then use.

In [100]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
s4 = Scoop('flavor 4')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')
s7 = Scoop('flavor 7')

class Bowl:
    MAX_SCOOPS = 3
    
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def add_scoops(self, *new_scoops):  
        for one_scoop in new_scoops:
            if len(self.scoops) >= Bowl.MAX_SCOOPS:  # I could also use self.MAX_SCOOPS here
                break
            self.add_scoop(one_scoop)   

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = 'Bowl with:\n'
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'  {index}: {one_scoop}\n'
        
        return output

b = Bowl()
b.add_scoops(s1, s2, s3)
b.add_scoops(s4, s5)
b.add_scoops(s6, s7)

print(b)

Bowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee



In [101]:
# what if I believe that class attributes are kind of "static," like in other languages?
# that would mean I can read/write the class attributes either using the class name or using self.
# does that work?

class Person:
    population = 0   

    def __init__(self, name):
        self.name = name
        Person.population += 1 

    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')
print(f'After, p1.population = {p1.population}')
print(f'After, p2.population = {p2.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 2
After, p1.population = 2
After, p2.population = 2
Hello, name1
Hello, name2


# The attribute lookup rules in Python

It is true that `population` is *ONLY* on the `Person` class, and is *not* on `p1` or `p2`. `population` is a class attribute on `Person`, and that means it is not on another object.

Unlike some other languges, there is *NO* way for objects to share attributes. An attribute is on object at a time.

However, if we ask an instance for an attribute, and the attribute isn't on the instance, Python doesn't give up. Instead, it goes to the instance's class and checks if the attribute exists there.

In other words:

- Python asks `p1`: Do you have an attribute named `population`? Answer: No.
- Python goes to `p1`'s class, `Person`, and asks: Do you have an attribute named `population`? Answer: Yes. We get the value back.

Why does Python do this?

For one, this way methods can work. Methods are attributes defined on the class. But we normally invoke them via the instance.

For example:

- We invoke `p1.greet`
- Python asks `p1` if it has an attribute `greet`. The answer: No.
- Python asks `p1`'s class, `Person`, if it has an attribute `greet`. The answer: Yes! We get the method back, and invoke it.

Whenever we see `a.b` in Python:

- Python asks `a` if it has an attribute `b`. If so, it returns the value
- If not, it asks `a`'s class if it has an attribute `b`. If so, it returns the value.

In [102]:
# given what we just said, what about the following code:


class Person:
    population = 0   

    def __init__(self, name):
        self.name = name
        self.population += 1    # won't this do the same thing?

    def greet(self):
        return f'Hello, {self.name}'

print(f'Before, population = {Person.population}')
p1 = Person('name1')
p2 = Person('name2')
print(f'After, population = {Person.population}')
print(f'After, p1.population = {p1.population}')
print(f'After, p2.population = {p2.population}')

print(p1.greet())
print(p2.greet())

Before, population = 0
After, population = 0
After, p1.population = 1
After, p2.population = 1
Hello, name1
Hello, name2


This is catastrophically bad! What happened?

When we say

    self.population += 1

in `__init__`, that gets translated into

    self.population = self.population + 1

1. First, the right side: Python asks `self` (the instance) if it has a `population` attribute. The answer: No.
2. Python asks `self`'s class, `Person`, if it has a `population` attribute. The answer: Yes! We get 0 back.
3. Then, we do the left side of assignment, setting `self.population` -- a new attribute on the instance! -- to be 1 (0 + 1)

The result is:
1. `p1` now has a new instance attribute, `population`, value 1.
2. `p2` now has a new instance attribute, `population`, value 1.
3. `Person` still has an attribute `population`, but its value has never changed from 0.

## Rules for easier living with attributes

1. It's totally fine to *retrieve* class attributes via `self` or the class name.
2. It's awful and dangerous to *set* class attributes via `self`. Only use the class name.

# Next up

1. The full ICPO (instance-class-parents-object) rule for attribute lookup
2. Inheritance
3. Moving forward with objects

# The story so far

- If we ask for an attribute on an instance
- If the attribute is *not* on the instance
- Python doesn't give up, but first checks on the class -- if it's there, we get the value back from there.

A major reason that things work this way is because methods are defined on the class. This allows us to invoke methods via the instance.

# Inheritance

One of the ideas in object-oriented programming is that we shouldn't have to reinvent the wheel. If I have defined a class, and you have a need that's very similar to what I did in my class, you should be able to reuse the parts of my class that are helpful to you.

This is known as *inheritance*:

- A parent class is defined
- A child class is defined that depends on that parent class
- In general, the child class defines where it is *different from* the parent
- Anything that the child class doesn't define for itself will be identical to the parent class

I think that people go **WAY** overboard in talking about inheritance. It is important! It is part of objects...

When should you use inheritance?

- If you want to write a class
- And there's already a class that implements much of the same functionality

So far, we've been using object *composition*, where one object owns another one. This is the "has-a" rule for composition, that one object "has" the other.

Inheritance, by contrast, uses the "is-a" rule. Can we say that the child class "is-a" version of the parent class?

- We can say that a `Bowl` has-a `Scoop`. Thus, we see a composition relationship.
- We can say that a `Car` is-a `Vehicle`. So if there is already a `Vehicle` class, we can probably take advantage of it and define a `Car` class that inherits from it.
- But we cannot say that a `Bowl` is-a `Scoop`. This makes no sense! Inheritance isn't appropriate here.

# Another story (related to the first)

We have our `Person` class, and our customers are happy. So happy that they want a new class, `Employee`, which is just like `Person` except that it also has a second attribute, `id_number`. How can we do this?

In [103]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1
Hello, name2


In [104]:
# a naive way to implement Employee: Copy and paste, and make some modifications!

class Employee:
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number

    def greet(self):
        return f'Hello, {self.name}'

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())
print(e2.greet())

Hello, emp1
Hello, emp2


Because I copied `Person` and then implemented `Employee`, I'm missing out:

- I had to write (well, paste) far more code than I should have
- If something changes (improves!) in `Person`, I won't benefit from it
- It's hard to read, maintain, and keep track of more code -- the less code, the better (in general)

Fortunately, we can restructure this to use inheritance.

Inheritance takes advantage of the full attribute lookup system:

- I -- first, check on the instance to see if the attribute is there. If so, great!
- C -- next, check on the instance's class to see if the attribute is there. If so, great!
- P -- check on the parent of the instance's class to see if it's there
- O -- Finally, check on `object`, the class that everyone inherits from in the end, the top of the hierarchy

In [107]:
class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())  # does p1 have greet? no! does Person have greet? Yes
print(p2.greet())  # same as above

class Employee(Person):    # the (Person) means: Employee is-a Person
    def __init__(self, name, id_number):
        self.name = name
        self.id_number = id_number

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())  # does e1 have greet? No. Does Employee have greet? No. Does Person have greet? Yes!
print(e2.greet())  # does e2 have greet? No. Does Employee have greet? No. Does Person have greet? Yes!

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


In [108]:
# it's really tempting to go even further!
# let's remove the assignment to self.name in Employee's __init__. After all, Person's __init__ sets it!

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())  # does p1 have greet? no! does Person have greet? Yes
print(p2.greet())  # same as above

class Employee(Person):    # the (Person) means: Employee is-a Person
    def __init__(self, name, id_number):
        self.id_number = id_number

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())  # does e1 have greet? No. Does Employee have greet? No. Does Person have greet? Yes!
print(e2.greet())  # does e2 have greet? No. Does Employee have greet? No. Does Person have greet? Yes!

Hello, name1!
Hello, name2!


AttributeError: 'Employee' object has no attribute 'name'

In [109]:
vars(e1)

{'id_number': 1}

In [110]:
vars(e2)

{'id_number': 2}

What's the problem?

- When we create a new instance of `Employee`, `__init__` runs. Which `__init__`? The first one that Python finds -- on `Employee`.
- This means that `Person.__init__` does *not* run.

In [111]:
# somehow, we're going to have to ensure that our Employee objects all have a "name" attribute
# option 1: Go back to setting it in __init__. Not the best solution
# option 2: Manually invoke Person.__init__ on any new instance of Employee

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())  # does p1 have greet? no! does Person have greet? Yes
print(p2.greet())  # same as above

class Employee(Person):    # the (Person) means: Employee is-a Person
    def __init__(self, name, id_number):
        Person.__init__(self, name)   # manually invoke __init__ on Person, to add name 
        self.id_number = id_number

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())  
print(e2.greet())  

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


In [112]:
# option 3: use "super" 
# super says: we're going to invoke a method from the parent class on our object

class Person:
    def __init__(self, name):
        self.name = name

    def greet(self):
        return f'Hello, {self.name}!'

p1 = Person('name1')
p2 = Person('name2')

print(p1.greet())  # does p1 have greet? no! does Person have greet? Yes
print(p2.greet())  # same as above

class Employee(Person):    # the (Person) means: Employee is-a Person
    def __init__(self, name, id_number):
        super().__init__(name)    # this means: invoke __init__ on whoever our parent is
        self.id_number = id_number

e1 = Employee('emp1', 1)
e2 = Employee('emp2', 2)

print(e1.greet())  
print(e2.greet())  

Hello, name1!
Hello, name2!
Hello, emp1!
Hello, emp2!


# Summary of inheritance so far

- We use inheritance when one class can rely on another class's implementation for most things (by default)
- If we don't define a method in our child class, then the parent class's method is invoked
- If we do define a method in our child class, then the parent class's method is *NOT* invoked
- If we want to combine our method with the parent class's method, then we can use `super()` to invoke the parent method.
- Normally, it's a good idea to invoke the parent method *first* and then do what we want in the child method.

In our above example:
- `Person` is the parent class, or superclass
- `Employee` is the child class, or subclass
- `Employee` doesn't define `greet`, so invoking `e1.greet()` runs `Person.greet`
- `Employee` *does* define `__init__`, so when we create a new instance of `Employee`, `Person.__init__` is not invoked by default.
- By putting a call to `super().__init__` in `Employee.__init__`, we ensure that the parent's `__init__` runs as well.

In [113]:
x = 5
x = 7

In [114]:
x

7

# Three paradigms for method inheritance

1. The child class does *not* implement the method. The parent class's method is invoked. This is a good default, because a child class should only implement things that are different. If you are overriding every method in the child, then why are you inheriting?
2. The child class *does* implement the method, and does things that are very different from the parent. Some examples might be how to display things.
3. The combo -- the child class has a method that does things not in the parent, *but* we use `super` to invoke the method in the parent as well, since we want the parent functionality but also to do some things our way.

# Exercise: `BigBowl`

1. We have a `Scoop` class, and a `Bowl` class. Currently, the maximum number of scoops we can put in an instance of `Bowl` is 3.
2. Implement a new class, `BigBowl`, which is identical to `Bowl`, except that it can hold up to 5 scoops at a time.
3. Define this new class, writing as little code as possible, and modifying as little code in `Bowl` as possible to get it to work.
4. Demonstrate that you can only have 3 scoops in a `Bowl` but 5 in a `BigBowl`.

In [123]:
class Scoop:
    def __init__(self, flavor):
        self.flavor = flavor
    def __repr__(self):
        return f'Scoop of {self.flavor}'

s1 = Scoop('chocolate')
s2 = Scoop('vanilla')
s3 = Scoop('coffee')
s4 = Scoop('flavor 4')
s5 = Scoop('flavor 5')
s6 = Scoop('flavor 6')
s7 = Scoop('flavor 7')

class Bowl:
    MAX_SCOOPS = 3
    
    def __init__(self):
        self.scoops = []

    def add_scoop(self, new_scoop):
        self.scoops.append(new_scoop)

    def flavors(self):
        return [one_scoop.flavor
                for one_scoop in self.scoops]

    def add_scoops(self, *new_scoops):  
        for one_scoop in new_scoops:
            if len(self.scoops) >= self.MAX_SCOOPS:  # I could also use self.MAX_SCOOPS here
                break
            self.add_scoop(one_scoop)   

    def __len__(self):
        return len(self.scoops)

    def __repr__(self):
        output = f'{type(self).__name__} with:\n'
        for index, one_scoop in enumerate(self.scoops, 1):
            output += f'  {index}: {one_scoop}\n'
        
        return output

class BigBowl(Bowl):
    MAX_SCOOPS = 5

b = Bowl()
b.add_scoops(s1, s2, s3)
b.add_scoops(s4, s5)
b.add_scoops(s6, s7)

print(b)
print('*****')
bb = BigBowl()
bb.add_scoops(s1, s2, s3)
bb.add_scoops(s4, s5)
bb.add_scoops(s6, s7)

print(bb)

Bowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee

*****
BigBowl with:
  1: Scoop of chocolate
  2: Scoop of vanilla
  3: Scoop of coffee
  4: Scoop of flavor 4
  5: Scoop of flavor 5



# `object`

At the top of our object hierarchy is `object`. Everything eventually inherits from it.

You can always check to see what a class inherits from by asking it, looking at its `__bases__` attribute.

This means that if an attribute isn't on an instance, isn't on a class, isn't on the class's parent, then we check on `object`. If it's there, then great -- and if not, we get an error.

`object` defines very few methods. But among them are `__str__` and `__repr__`. This is why every object in Python knows how to turn itself into a string, why every object in Python can be printed, and also why every object in Python has a default string value that is super ugly.

In [124]:
BigBowl.__bases__

(__main__.Bowl,)

In [125]:
Bowl.__bases__

(object,)

# A few points

1. Projects and practice
    - Find a programming project, and think about the data structures you'll want. What will make programming it easier? What will make the most important things stick out and become obvious?
    - Join an open-source project that uses objects (which is most of them)
    - As your projects get larger, you'll need to consider more and more how many objects you have, and their relationships
    - As you do projects in Python, you'll use objects -- look at the ones you use, how they're implemented, and what works and what doesn't.
2. More advanced ideas and practice
    - Magic methods we didn't implement or play with
    - Multiple inheritance
    - Object system -- `type` vs. `object`
    - Metaclasses (you probably don't need these)
    - Properties -- which let you have the feeling that you're using data, but really you're invoking methods
    - Descriptors -- which are how properties and methods are implemented